<a href="https://colab.research.google.com/github/Stefano-t/bioinf-lab/blob/main/ae_vs_ie_ffnn_cnn_mmnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print("installing dependency...")
!wget "https://raw.githubusercontent.com/LucaCappelletti94/bioinformatics_practice/master/requirements.txt" &> /dev/null
%pip install -r requirements.txt &> /dev/null
#print("downloading epigenomic_dataset...")
#%pip install epigenomic_dataset &> /dev/null && echo "epigenomic_dataset installed!"
print("downloading humanize 3.6.0...")
%pip install -Iv humanize==3.6.0 &> /dev/null && echo "humanize 3.6.0 installed!"
print("downloading extra_keras_metrics...")
%pip install extra_keras_metrics &> /dev/null && echo "extra_keras_metrics installed!"

!rm requirements.txt
print("finish!")

installing dependency...
downloading humanize 3.6.0...
humanize 3.6.0 installed!
downloading extra_keras_metrics...
extra_keras_metrics installed!
finish!


In [10]:
%tensorflow_version 2.x
import tensorflow as tf
from epigenomic_dataset import active_enhancers_vs_inactive_enhancers
from epigenomic_dataset.utils import normalize_epigenomic_data

from sklearn.impute import KNNImputer
import pandas as pd
from sklearn.preprocessing import RobustScaler
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm.auto import tqdm
from tqdm.keras import TqdmCallback

from sklearn.tree import DecisionTreeClassifier
from cache_decorator import Cache

from typing import Dict, List
from sanitize_ml_labels import sanitize_ml_labels
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score

from keras_mixed_sequence import MixedSequence, VectorSequence
from keras_bed_sequence import BedSequence
from ucsc_genomes_downloader import Genome

In [ ]:
genome = Genome("hg38")

In [15]:
cell_line = "K562"
window_size = 256

X, y = active_enhancers_vs_inactive_enhancers(
    cell_line = cell_line,
    window_size=window_size,
)

bed_X = X.reset_index()
bed = bed_X[bed_X.columns[:5]]

In [16]:
X

chrom                              SMAD5  NCOA2  ZNF23  ...  NCOR1  NCOA1  KLF1
chrom chromStart chromEnd  strand                       ...                    
chr10 100006381  100006637 .        0.90   0.90   1.37  ...   2.38   0.82  0.69
      100008146  100008402 .        0.73   1.06   0.74  ...   3.11   0.49  1.05
      100014418  100014674 .        0.77   0.33   0.53  ...   0.61   0.97  0.96
      100020216  100020472 .        0.87   0.80   0.83  ...   0.57   0.03  1.07
      100043528  100043784 .        0.64   0.53   0.03  ...   1.26   1.73  0.33
...                                  ...    ...    ...  ...    ...    ...   ...
chrY  7520247    7520503   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7724272    7724528   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7770029    7770285   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      7796295    7796551   .        0.00   0.00   0.00  ...   0.00   0.00  0.00
      8007562    8007818   .        0.00   0.00   0.00  ...   0.00   0.00  0.00

[63285 rows x 429 columns]

In [18]:
# def build_sequence(
#     X: pd.DataFreame,
#     y: np.ndarray,
#     genome: Genome,
#     batch_size: int
# ) -> MixedSequence:

def get_mmnn_sequence(
    genome: Genome,
    bed: pd.DataFrame,
    X: np.ndarray,
    y: np.ndarray,
    batch_size: int = 256
) -> MixedSequence:
    return MixedSequence(
        x = {
            "sequence_data": BedSequence(
                genome,
                bed,
                batch_size = batch_size,
            ),
            "epigenomic_data": VectorSequence(
                X.values,
                batch_size
            )
        },
        y = VectorSequence(
            y.values,
            batch_size = batch_size,
        )
    )

def get_ffnn_sequence(
    X: np.ndarray,
    y: np.ndarray,
    batch_size: int = 256
) -> MixedSequence:
    return MixedSequence(
        x = {
            "epigenomic_data": VectorSequence(
                X.values,
                batch_size
            )
        },
        y = VectorSequence(
            y.values,
            batch_size = batch_size,
        )
    )

def get_cnn_sequence(
    genome: Genome,
    bed: pd.DataFrame,
    y: np.ndarray,
    batch_size: int = 256
) -> MixedSequence:
    return MixedSequence(
        x = {
            "sequence_data": BedSequence(
                genome,
                bed,
                batch_size = batch_size,
            )
        },
        y = VectorSequence(
            y.values,
            batch_size = batch_size,
        )
    )

In [21]:
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, ReLU, Concatenate, Layer
from tensorflow.keras.layers import Conv1D,  MaxPool1D, GlobalAveragePooling1D, GlobalMaxPool1D, Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from extra_keras_metrics import get_complete_binary_metrics

In [20]:
# ffnn
def build_ffnn(X: pd.DataFrame):
    input_epigenomic_data = Input(
        shape=(X.shape[1]),
        name="epigenomic_data",
    )

    hidden = Dense(
        128,
        # use linear activation because the step of BatchNormalization
        # doesn't work properly with ReLU, due to the zeros in output
        activation="linear"
    ) (input_epigenomic_data)

    hidden = BatchNormalization() (hidden)
    hidden = ReLU() (hidden)

    for _ in range(3):
        hidden = Dense(
            64,
            activation="relu",
            kernel_regularizer=regularizers.l1_l2(),
            activity_regularizer=regularizers.l1_l2(),
        ) (hidden)
        hidden = Dropout(rate=0.3) (hidden)

    last_hidden_ffnn = hidden

    output_ffnn = Dense(
        1,
        activation="sigmoid"
    ) (last_hidden_ffnn)

    ffnn = Model(
        inputs=input_epigenomic_data,
        outputs=output_ffnn
    )

    ffnn.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=get_complete_binary_metrics()
    )

    return ffnn, input_epigenomic_data, last_hidden_ffnn 

In [19]:
# cnn
def build_cnn(window_size: int):
    input_sequence_data = Input(
        shape=(window_size, 4),
        name="sequnce_data",
    )

    hidden = Conv1D(
        64,
        kernel_size=6,
        # use linear activation because the step of BatchNormalization
        # doesn't work properly with ReLU, due to the zeros in output
        activation="linear"
    ) (input_sequence_data)

    hidden = BatchNormalization() (hidden)
    hidden = ReLU() (hidden)

    for _ in range(3):
        hidden = Conv1D(
            32,
            kernel_size=4,
            activation="relu",
            kernel_regularizer=regularizers.l1_l2(),
            activity_regularizer=regularizers.l1_l2(),
            # strides=2
        ) (hidden)
        hidden = Dropout(rate=0.3) (hidden)
        hidden = MaxPool1D(pool_size=2) (hidden)

    # hidden = Flatten() (hidden)
    # hidden = GlobalMaxPool1D() (hidden)
    hidden = GlobalAveragePooling1D() (hidden)

    last_hidden_cnn = Dense(
        64,
        activation="relu",
    ) (hidden)

    output_cnn = Dense(
        1,
        activation="sigmoid"
    ) (last_hidden_cnn) 

    cnn = Model(
        inputs=input_sequence_data,
        outputs=output_cnn,
    )

    cnn.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=get_complete_binary_metrics()
    )

    return cnn, input_sequence_data, last_hidden_cnn

In [23]:
# mmnn
def build_mmnn(
    X: pd.DataFrame = None,
    window_size: int = None,
    input_sequence_data: Layer = None,
    input_epigenomic_data: Layer = None,
    last_hidden_cnn: Layer = None,
    last_hidden_ffnn: Layer = None,
):
    if X is not None:
        _, input_epigenomic_data, last_hidden_ffnn = build_ffnn(X)
    if window_size is not None:
        _, input_sequence_data, last_hidden_cnn = build_cnn(window_size)
    
    concatenation_layer = Concatenate() ([
        last_hidden_ffnn,
        last_hidden_cnn,
    ])

    last_hidden_mmnn = Dense(
        64,
        activation="relu",
    ) (concatenation_layer)

    output_mmnn = Dense(
        1,
        activation="sigmoid"
    ) (last_hidden_mmnn) 


    mmnn = Model(
        inputs=[input_epigenomic_data, input_sequence_data],
        outputs=output_mmnn
    )

    mmnn.compile(
        optimizer="nadam",
        loss="binary_crossentropy",
        metrics=get_complete_binary_metrics()
    )

    return mmnn

In [25]:
@Cache(
    cache_path=[
        "active_enhancers_performance/{model_name}/history_{_hash}.csv.xz",
        "active_enhancers_performance/{model_name}/performance_{_hash}.csv.xz",
    ],
    args_to_ignore=[
        "model", "training_sequence", "test_sequence"
    ]
)
def train_model(
    model: Model,
    model_name: str,
    training_sequence: MixedSequence,
    test_sequence: MixedSequence,
    holdout_number: int
):
    history = pd.DataFrame(model.fit(
        training_sequence,
        validation_data=test_sequence,
        epochs=1000,
        verbose=False,
        callbacks=[
                   EarlyStopping("loss"),
                   TqdmCallback(verbose=1)
        ]
    ).history)

    train_evaluation = dict(zip(model.metrics_names, model.evaluate(training_sequence, verbose=False)))
    test_evaluation = dict(zip(model.metrics_names, model.evaluate(test_sequence, verbose=False)))
    train_evalutaion["run_type"] = "train"
    test_evalutation["run_type"] = "test"

    for evaluation in (train_evaluation, test_evalutation):
        evaluation["model_name"] = model_name
        evaluation["holdout_number"] = holdout_number

    evaluations = pd.DataFrame([
                                train_evaluation,
                                test_evaluation
    ])

    return history, evaluations

In [26]:
number_of_splits = 2

holdouts_generator = StratifiedShuffleSplit(
  n_splits = number_of_splits,
  test_size = 0.2, # 20% of the dataset as test set
)

In [28]:
all_performance = []

for train_indices, test_indices in tqdm(
    holdouts_generator.split(X, y),
    total=number_of_splits,
    desc="Computing holdouts"
):
    train_bed, test_bed = bed.iloc[train_indices], bed.iloc[test_indices]
    train_X, test_X = X.iloc[train_indices], X.iloc[test_indices]
    train_y, test_y = y.iloc[train_indices], y.iloc[test_indices]
    
    train_X, test_X = train_X.values, test_X.values
    train_y = train_y.values.flatten()
    test_t = test_y.values.flatten()

    ffnn, input_epigenomic_data, last_hidden_ffnn = build_ffnn(train_X)
    cnn, input_sequence_data, last_hidden_cnn = build_cnn(window_size)
    mmnn1 = build_mmnn(train_X, window_size)
    # this model uses the same weigths of ffnn and cnn models
    mmnn2 = build_mmnn(
        input_sequence_data=input_sequence_data,
        input_epigenomic_data=input_epigenomic_data,
        last_hidden_cnn=last_hidden_cnn,
        last_hidden_ffnn=last_hidden_ffnn
    )

    for model, train_sequence, test_sequence in tqdm(
        (
            (ffnn, get_ffnn_sequence(train_X, train_y), get_ffnn_sequence(test_X, test_y)),
            (cnn, get_cnn_sequence(genome, train_bed, train_y), get_cnn_sequence(genome, test_bed, test_y)),
            (mmnn1, get_mmnn_sequence(genome, train_bed, train_X, train_y), get_mmnn_sequence(genome, test_bed, test_X, test_y)),
            (mmnn2, get_mmnn_sequence(genome, train_bed, train_X, train_y), get_mmnn_sequence(genome, test_bed, test_X, test_y))
        ),
        desc="Training models",
        leave=False
    ):

        history, performance = train_model(model, train_sequence, test_sequence)
        all_performance.append(performance)

all_performance = pd.DataFrame(all_performance)

AttributeError: ignored

pandas.core.frame.DataFrame